In [1]:
" Code snippets from "
" https://www.codeproject.com/Articles/1237557/Two-Ways-to-Implement-LSTM-Network-using-Python-wi "

import numpy as np
import collections

class DataHandler:
    def read_data(self, fname):
        with open(fname) as f:
            content = f.readlines()
        content = [x.strip() for x in content]
        content = [content[i].split() for i in range(len(content))]
        content = np.array(content)
        content = np.reshape(content, [-1, ])
        return content
    
    def build_datasets(self, words):
        count = collections.Counter(words).most_common()
        dictionary = dict()
        for word, _ in count:
            dictionary[word] = len(dictionary)
        reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
        return dictionary, reverse_dictionary

In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn

class RNNGenerator:
    def create_LSTM(self, inputs, weights, biases, seq_size, num_units):
        # Reshape input to [1, sequence_size] and split it into sequences
        inputs = tf.reshape(inputs, [-1, seq_size])
        inputs = tf.split(inputs, seq_size, 1)
    
        # LSTM with 2 layers
        rnn_model = rnn.MultiRNNCell([rnn.BasicLSTMCell(num_units),rnn.BasicLSTMCell(num_units)])
    
        # Generate prediction
        outputs, states = rnn.static_rnn(rnn_model, inputs, dtype=tf.float32)
    
        return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [8]:
import random

class SessionRunner():
    training_iters = 20000
            
    def __init__(self, optimizer, accuracy, cost, lstm, initilizer, writer):
        self.optimizer = optimizer
        self.accuracy = accuracy
        self.cost = cost
        self.lstm = lstm
        self.initilizer = initilizer
        self.writer = writer
    
    def run_session(self, x, y, n_input, dictionary, reverse_dictionary, training_data, test_data):
        
        with tf.Session() as session:
            session.run(self.initilizer)
            step = 0
            offset = random.randint(0, n_input + 1)
            acc_total = 0
        
            self.writer.add_graph(session.graph)
        
            while step < self.training_iters:
                if offset > (len(training_data) - n_input - 1):
                    offset = random.randint(0, n_input+1)
        
                sym_in_keys = [ [dictionary[ str(training_data[i])]] 
                                 for i in range(offset, offset+n_input) ]
                sym_in_keys = np.reshape(np.array(sym_in_keys), [-1, n_input, 1])
        
                sym_out_onehot = np.zeros([len(dictionary)], dtype=float)
                sym_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
                sym_out_onehot = np.reshape(sym_out_onehot,[1,-1])
                
                _, acc, loss, onehot_pred = session.run([self.optimizer, self.accuracy, 
                   self.cost, self.lstm], feed_dict={x: sym_in_keys, y: sym_out_onehot})
                acc_total += acc
                
                if (step + 1) % 1000 == 0:
                    print("Iteration = " + str(step + 1) + ", Average Accuracy= " + 
                                           "{:.2f}%".format(100*acc_total/1000))
                    offset_t = -n_input
                    sym_in_keys_test = [ [dictionary[ str(test_data[i])]] 
                                     for i in range(offset_t, offset_t+n_input) ]
                    sym_in_keys_test = np.reshape(np.array(sym_in_keys_test), [-1, n_input, 1])

                    sym_out_onehot_test = np.zeros([len(dictionary)], dtype=float)
                    sym_out_onehot_test[dictionary[str(training_data[offset_t+n_input])]] = 1.0
                    sym_out_onehot_test = np.reshape(sym_out_onehot_test,[1,-1])
                    _, _, _, onehot_pred_test = session.run([self.optimizer, self.accuracy, 
                                                             self.cost, self.lstm], feed_dict={x: sym_in_keys_test, y: sym_out_onehot_test})
                    print(np.argmax(onehot_pred_test))
                    acc_total = 0
                step += 1
                offset += (n_input+1)
                
                
                
    def run_session_open_ended(self, x, y, n_input, dictionary, reverse_dictionary, training_data, test_data):
        
        session = tf.Session()
        session.run(self.initilizer)
        step = 0
        offset = random.randint(0, n_input + 1)
        acc_total = 0

        self.writer.add_graph(session.graph)

        while step < self.training_iters:
            if offset > (len(training_data) - n_input - 1):
                offset = random.randint(0, n_input+1)

            sym_in_keys = [ [dictionary[ str(training_data[i])]] 
                             for i in range(offset, offset+n_input) ]
            sym_in_keys = np.reshape(np.array(sym_in_keys), [-1, n_input, 1])

            sym_out_onehot = np.zeros([len(dictionary)], dtype=float)
            sym_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
            sym_out_onehot = np.reshape(sym_out_onehot,[1,-1])

            _, acc, loss, onehot_pred = session.run([self.optimizer, self.accuracy, 
               self.cost, self.lstm], feed_dict={x: sym_in_keys, y: sym_out_onehot})
            acc_total += acc

            if (step + 1) % 1000 == 0:
                print("Iteration = " + str(step + 1) + ", Average Accuracy= " + 
                                       "{:.2f}%".format(100*acc_total/1000))
                offset_t = -n_input
                sym_in_keys_test = [ [dictionary[ str(test_data[i])]] 
                                 for i in range(offset_t, offset_t+n_input) ]
                sym_in_keys_test = np.reshape(np.array(sym_in_keys_test), [-1, n_input, 1])

                sym_out_onehot_test = np.zeros([len(dictionary)], dtype=float)
                sym_out_onehot_test[dictionary[str(training_data[offset_t+n_input])]] = 1.0
                sym_out_onehot_test = np.reshape(sym_out_onehot_test,[1,-1])
                onehot_pred_test = session.run([self.lstm], feed_dict={x: sym_in_keys_test, y: sym_out_onehot_test})
                print(np.argmax(onehot_pred_test))
                acc_total = 0
            step += 1
            offset += (n_input+1)
            
        return session, [self.optimizer, self.accuracy, self.cost, self.lstm]

In [6]:
log_path = '/output/tensorflow/'
writer = tf.summary.FileWriter(log_path)

# Load and prepare data
data_handler = DataHandler()

training_data =  flatten(data_handler.read_data('Ch1.tex'))
#test_data =  data_handler.read_data('test.txt')
test_data = training_data

dictionary, reverse_dictionary = data_handler.build_datasets(training_data)

# TensorFlow Graph input
n_input = 5
n_units = 512

x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, len(dictionary)])

# RNN output weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_units, len(dictionary)]))
}
biases = {
    'out': tf.Variable(tf.random_normal([len(dictionary)]))
}

rnn_generator = RNNGenerator()
lstm = rnn_generator.create_LSTM(x, weights, biases, n_input, n_units)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=lstm, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(lstm,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
initilizer = tf.global_variables_initializer()

session_runner = SessionRunner(optimizer, accuracy, cost, lstm, initilizer, writer)

In [9]:
session_runner = SessionRunner(optimizer, accuracy, cost, lstm, initilizer, writer)
saved_sess, sess_parameters = session_runner.run_session_open_ended(x, y, n_input, dictionary, reverse_dictionary, 
                                                   training_data, test_data)

Iteration = 1000, Average Accuracy= 4.00%
1
Iteration = 2000, Average Accuracy= 6.10%
8
Iteration = 3000, Average Accuracy= 5.00%
1
Iteration = 4000, Average Accuracy= 6.20%
3
Iteration = 5000, Average Accuracy= 5.60%
3
Iteration = 6000, Average Accuracy= 5.30%
5
Iteration = 7000, Average Accuracy= 6.00%
1
Iteration = 8000, Average Accuracy= 6.00%
1
Iteration = 9000, Average Accuracy= 6.40%
1
Iteration = 10000, Average Accuracy= 6.80%
2
Iteration = 11000, Average Accuracy= 6.00%
1
Iteration = 12000, Average Accuracy= 5.20%
0
Iteration = 13000, Average Accuracy= 6.10%
1
Iteration = 14000, Average Accuracy= 6.60%
1
Iteration = 15000, Average Accuracy= 5.70%
0
Iteration = 16000, Average Accuracy= 6.80%
8
Iteration = 17000, Average Accuracy= 5.70%
7
Iteration = 18000, Average Accuracy= 5.70%
8
Iteration = 19000, Average Accuracy= 6.80%
7
Iteration = 20000, Average Accuracy= 6.20%
8


In [ ]:
test_data =  data_handler.read_data('test3.txt')
offset_t = -n_input
sym_in_keys_test = [ [dictionary[ str(test_data[i])]] 
                 for i in range(offset_t, offset_t+n_input) ]
sym_in_keys_test = np.reshape(np.array(sym_in_keys_test), [-1, n_input, 1])

sym_out_onehot_test = np.zeros([len(dictionary)], dtype=float)
sym_out_onehot_test[dictionary[str(training_data[offset_t+n_input])]] = 1.0
sym_out_onehot_test = np.reshape(sym_out_onehot_test,[1,-1])
#_, _, _, onehot_pred_test = saved_sess.run(sess_parameters, feed_dict={x: sym_in_keys_test, y: sym_out_onehot_test})
onehot_pred_test = saved_sess.run(sess_parameters[-1], feed_dict={x: sym_in_keys_test, y: sym_out_onehot_test})
print(reverse_dictionary[np.argmax(onehot_pred_test)])
print(np.argmax(onehot_pred_test))

In [ ]:
iterations = 10
test_data =  data_handler.read_data('prompt.txt')
offset_t = -n_input
sym_in_keys_test = [ [dictionary[ str(test_data[i])]] 
                 for i in range(offset_t, offset_t+n_input) ]
sym_in_keys_test = np.reshape(np.array(sym_in_keys_test), [-1, n_input, 1])
#Printing (end of) prompt
for j in range(n_input): print(reverse_dictionary[sym_in_keys_test[0][j][0]])

sym_out_onehot_test = np.zeros([len(dictionary)], dtype=float)
sym_out_onehot_test[dictionary[str(training_data[offset_t+n_input])]] = 1.0
sym_out_onehot_test = np.reshape(sym_out_onehot_test,[1,-1])

#First prediction
onehot_pred_test = saved_sess.run(sess_parameters[-1], feed_dict={x: sym_in_keys_test, y: sym_out_onehot_test})
prediction = np.argmax(onehot_pred_test)
print(reverse_dictionary[prediction])

#Continued predictions
for i in range(iterations):
    sym_in_keys_new = np.append(sym_in_keys_test,prediction)
    sym_in_keys_test = np.reshape(np.array(sym_in_keys_new[-n_input:]), [-1, n_input, 1])
    
    #This can be whatever
    sym_out_onehot_test = np.zeros([len(dictionary)], dtype=float)
    sym_out_onehot_test[dictionary[str(training_data[offset_t+n_input])]] = 1.0
    sym_out_onehot_test = np.reshape(sym_out_onehot_test,[1,-1])

    #Further predictions
    onehot_pred_test = saved_sess.run(sess_parameters[-1], feed_dict={x: sym_in_keys_test, y: sym_out_onehot_test})
    prediction = np.argmax(onehot_pred_test)
    print(reverse_dictionary[np.argmax(onehot_pred_test)])

In [ ]:
A = [list(['\\chapter{Overview}']), list([]),
 list(['The', 'concept', 'of', 'quantum', 'computers', 'was', 'first', 'introduced', 'by', 'Richard', 'Feynman', '\\cite{Feynman82},', 'in', 'the', 'context', 'of', 'using', 'a', 'quantum', 'system', 'to', 'simulate', 'quantum', 'phenomena.', 'He', 'believed', 'that', "``there's", 'plenty', 'of', 'room', 'at', 'the', "bottom''", '\\cite{Feynman60},', 'and', 'he', 'was', 'right.', 'Since', 'then,', 'the', 'field', 'of', 'quantum', 'computation', 'underwent', 'an', 'enormous', 'development:', 'from', 'when', 'David', 'Deutsch', 'demonstrated', 'the', 'power', 'of', '``parallel', 'computing', 'with', "qubits''", '\\cite{Deutsch85},', 'when', 'Peter', 'Shor', 'constructed', 'his', 'factoring', 'algorithm', '\\cite{Shor94},', 'to', 'the', 'first', 'concept', 'of', 'quantum', 'error-correcting', 'codes', '(also', 'pioneered', 'by', 'Shor', '\\cite{CS96,Steane96,Gottesman97}),', 'all', 'the', 'way', 'to', 'today,', 'when', 'several', 'research', 'groups,', 'both', 'academic', 'and', 'commercial,', 'already', 'have', 'or', 'are', 'on', 'the', 'brink', 'of', 'realizing', 'working', 'quantum', 'computers', '\\cite{MRN+17,Castelvecchi17}.']),
 list([])]

In [5]:
def flatten(l):
    merged = []
    for el in l:
        merged = merged + el
    return merged

In [ ]:
flatten(A)